In [3]:
from sympy import symbols, diff, Function, Eq, solve, collect, prod

t,V,f,k,CA0,V1,V2 = symbols("t,V,f,k,C_A0,V1,V2",positive=True)
CA1,CA2 = symbols('C_A1,C_A2',cls=Function)

def var_in_part(var,part):
    # Read the next few nodes
    _ = sum(map(lambda x:x.as_ordered_factors(),part.as_ordered_terms()),start=[])

    # If not at a leaf search further, if at leaf check if var is variable
    if len(_) > 1:
        return any(var_in_part(var,part) for part in _)
    elif len(_) == 1:
        return _[0] == var
    else:
        raise NotImplementedError()

def to_standard_deqn(E,y,x,t):
    '''
    Brings a differential equation to standard form
    `tau * dy/dt + y = K * x`
    returns `tau` and `K`
    '''
    # Isolate the x function
    sol = solve(E,x)
    assert len(sol) == 1
    
    # Save cofactors for later
    col = collect(sol[0],y)
    *co_factor,col = col.as_ordered_factors()

    # Split into dy and y
    terms = col.as_ordered_terms()
    assert len(terms) == 2
    _dy,_y = terms if var_in_part(y,terms[1]) else terms[::-1]
    assert var_in_part(y,_y)
    assert var_in_part(diff(y,t),_dy)

    # Extract coefficients
    *K_inv,_y = [1]+_y.as_ordered_factors()
    *tau_K_inv,_dy = [1]+_dy.as_ordered_factors()
    assert y == _y
    assert diff(y,t) == _dy

    # Combine coefficients
    K_inv     = prod(co_factor+K_inv)
    tau_K_inv = prod(co_factor+tau_K_inv)
    tau = tau_K_inv/K_inv
    K = 1/K_inv

    return tau,K

def from_standard_deqn(y,x,t,tau,K):
    '''
    Constructs differential equation on standard form
    `tau * dy/dt + y = K * x`
    given `tau` and `K`
    '''
    return Eq(tau*diff(y,t) + y, K * x)

deqn_1 = Eq(diff(V1*CA1(t),t), f*CA0   -f*CA1(t)-V1*k*CA1(t))
deqn_2 = Eq(diff(V2*CA2(t),t), f*CA1(t)-f*CA2(t)-V2*k*CA2(t))
tau1,K1 = to_standard_deqn(deqn_1,CA1(t),CA0   ,t)
tau2,K2 = to_standard_deqn(deqn_2,CA2(t),CA1(t),t)
assert tau1 == V1/(f+V1*k)
assert K1 == f/(f+V1*k)
assert tau2 == V2/(f+V2*k)
assert K2 == f/(f+V2*k)

display ('this is eq 1', deqn_1)
display ('this is Tau1 and K1', tau1, K1)
display ('this is eq 2', deqn_2)
display ('this is Tau2 and K2', tau2, K2)

'this is eq 1'

Eq(V1*Derivative(C_A1(t), t), C_A0*f - V1*k*C_A1(t) - f*C_A1(t))

'this is Tau1 and K1'

V1/(V1*k + f)

f/(V1*k + f)

'this is eq 2'

Eq(V2*Derivative(C_A2(t), t), -V2*k*C_A2(t) + f*C_A1(t) - f*C_A2(t))

'this is Tau2 and K2'

V2/(V2*k + f)

f/(V2*k + f)